<a href="https://colab.research.google.com/github/LuanDavid7/AI_EKT/blob/main/AI_Answer_EKT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CÓDIGO ATUAL**

**Instala Bibliotca faltante**

In [ ]:
!pip install python-dotenv

**Código Inicial Final**


Versão com Totos os problemas cadastrados

In [ ]:
import requests
import json
import datetime
import re
import os
import pytz
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

# Configurações da API
API_KEY = os.getenv("GROQ_API_KEY", "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")
MODEL = "llama3-70b-8192"
URL = "https://api.groq.com/openai/v1/chat/completions"

# Configurar fuso horário de Brasília
TIMEZONE = pytz.timezone('America/Sao_Paulo')

# Base de conhecimento da Neoenergia Elektro
CATEGORIAS_PROBLEMAS = [
    {"categoria": "Ocorrência Técnica", "tipo": "Emergencia médica", "subtipo": "EMERGENCIA MEDICA - EQUIPAMENTO SUPORTE A VIDA", "criticidade": "Emergência", "termos": ["Tenho equipamento de suporte à vida", "Estou com respirador", "Minha máquina de oxigênio parou"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Emergencia médica", "subtipo": "EMERGENCIA MEDICA - HOSPITAL COM PROBLEMA", "criticidade": "Emergência", "termos": ["O hospital está sem luz", "Preciso de energia para um hospital"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Energia - Geral", "criticidade": "Médio", "termos": ["Minha casa está sem luz", "Só aqui em casa que caiu a energia"], "interno": True},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Energia em Medidor Eletrônico", "criticidade": "Médio", "termos": ["Meu relógio digital está apagado", "O medidor novo não está funcionando"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Energia Geral - Área Amplia", "criticidade": "Alto", "termos": ["Faltou luz na rua", "A vizinhança inteira sem energia", "Apagão"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "FALTA DE FASE", "criticidade": "Médio", "termos": ["Algumas lâmpadas acendem, outras não", "Só um lado da casa tem luz"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Fase em Medidor Eletrônico", "criticidade": "Médio", "termos": ["Meu medidor está piscando em vermelho", "Aparece um erro no painel do relógio"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "SIGFI FALTA DE ENERGIA FOTOVOLTAICO", "criticidade": "Médio", "termos": ["Meu sistema solar não está gerando energia", "As baterias não estão carregando"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Níveis de tensão", "subtipo": "OSCILACÃO DE TENSÃO", "criticidade": "Médio", "termos": ["A luz fica piscando", "Meus aparelhos desligam e ligam sozinhos"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Níveis de tensão", "subtipo": "TENSÃO ALTA", "criticidade": "Alto", "termos": ["A luz está muito forte", "Meus aparelhos estão queimando"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Níveis de tensão", "subtipo": "TENSÃO BAIXA", "criticidade": "Médio", "termos": ["A luz está fraca", "Meus aparelhos não funcionam direito"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Outros", "subtipo": "TRANSFORMADOR", "criticidade": "Alto", "termos": ["O transformador está fazendo barulho", "Tem um cheiro estranho vindo do poste"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "CHOQUE EM INSTALAÇÃO", "criticidade": "Emergência", "termos": ["Estou tomando choque na torneira", "A geladeira está dando choque"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "ELO FUSIVEL QUEIMADO/CAIDO", "criticidade": "Alto", "termos": ["Tem algo pendurado no poste", "Um dispositivo caiu do transformador"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FALHA EM MEDIDOR", "criticidade": "Médio", "termos": ["Meu relógio de luz está com problema", "O medidor parece quebrado"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FALHA EM QUICK-LAG", "criticidade": "Médio", "termos": ["O disjuntor do poste parece queimado", "Algo explodiu na caixa do poste"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FALHA EM RAMAL DE SERVICO", "criticidade": "Alto", "termos": ["O fio que vem para minha casa está solto", "A conexão com o poste está com problema"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO COM ARVORE NA REDE", "criticidade": "Emergência", "termos": ["Está saindo faísca do fio com a árvore", "A árvore está pegando fogo no fio"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO NA REDE PRIMARIA", "criticidade": "Emergência", "termos": ["Está saindo fogo dos fios de alta tensão", "Os cabos grossos estão soltando faíscas"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO NA REDE SECUNDARIA", "criticidade": "Alto", "termos": ["O fio da rua está soltando faísca", "Está saindo fogo do cabo de luz"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO NO RAMAL DE SERVICO", "criticidade": "Alto", "termos": ["O fio que entra na minha casa está pegando fogo", "Tem faísca na entrada de luz"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FURTO EQUIPAMENTO", "criticidade": "Alto", "termos": ["Roubaram algo do poste", "Levaram equipamento da rede"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FURTO FIACÃO/PADRÃO", "criticidade": "Alto", "termos": ["Roubaram os fios da minha casa", "Levaram o medidor"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FURTO REDE", "criticidade": "Alto", "termos": ["Roubaram os fios da rua", "Tem cabos cortados no poste"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "INVERSÃO DE FASE", "criticidade": "Médio", "termos": ["Meu motor está girando ao contrário", "O equipamento trifásico não funciona direito"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "OBJETO ESTRANHO NA REDE PRIMARIA", "criticidade": "Alto", "termos": ["Tem algo preso nos fios de alta tensão", "Um objeto está encostado na rede"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "OBJETO ESTRANHO NA REDE SECUNDARIA", "criticidade": "Médio", "termos": ["Jogaram algo nos fios", "Tem um objeto preso na fiação"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "POSTE PADRÃO QUEBRADO", "criticidade": "Alto", "termos": ["Meu poste padrão está quebrado", "A entrada de luz está torta"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "FIO PARTIDO NA REDE PRIMARIA", "criticidade": "Emergência", "termos": ["O fio grosso da alta tensão está partido", "Caiu um cabo de alta tensão"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "FIO PARTIDO NA REDE SECUNDARIA", "criticidade": "Emergência", "termos": ["O fio da luz caiu", "Tem um cabo quebrado na rua"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "FIO PARTIDO NO RAMAL DE SERVICO", "criticidade": "Alto", "termos": ["O fio que vem para minha casa está quebrado", "Caiu o cabo da entrada"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "POSTE ABALROADO COM FIO PARTIDO", "criticidade": "Emergência", "termos": ["Alguém bateu no poste e o fio caiu", "Poste quebrado com fios soltos"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "POSTE ABALROADO SEM FIO PARTIDO", "criticidade": "Alto", "termos": ["O poste está torto depois do acidente", "Bateram no poste mas os fios estão intactos"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Ligação", "subtipo": "Ligação Nova", "criticidade": "Médio", "termos": ["Preciso de uma nova ligação de energia", "Minha casa ainda não tem luz"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Religação", "subtipo": "Religação Normal", "criticidade": "Médio", "termos": ["Paguei a conta e a luz não voltou", "Quero religar a energia"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Religação", "subtipo": "Religação de Urgência", "criticidade": "Alto", "termos": ["Preciso da energia hoje mesmo", "Tenho uma pessoa doente em casa"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Desligamento progamado", "subtipo": "Desligamento para Manutenção", "criticidade": "Médio", "termos": ["Vai faltar luz aqui?", "Tem alguma manutenção programada?"], "interno": False}
]

def obter_saudacao():
    """Retorna uma saudação baseada no horário de Brasília."""
    try:
        hora_atual = datetime.datetime.now(TIMEZONE).hour
        if 5 <= hora_atual < 12:
            return "Bom dia"
        elif 12 <= hora_atual < 18:
            return "Boa tarde"
        else:
            return "Boa noite"
    except Exception:
        return "Olá"

def mensagem_fechamento():
    """Retorna a mensagem padrão de fechamento."""
    return """Agradecemos seu contato! Sempre que precisar estamos disponíveis nos nossos canais:

💻 Site: https://www.neoenergia.com/web/sp

📱 Aplicativo:
Android → https://bit.ly/NELK-Android
IOS → https://bit.ly/IOS-NELK

📞 Teleatendimento: 116
Deficiente auditivo ou de fala | 0800 701 01 55
Atendimento fora da área de concessão | 0800 276 0116
Conte sempre com a gente!"""

def gerar_protocolo():
    """Gera um número de protocolo baseado na data e hora atuais em Brasília."""
    agora = datetime.datetime.now(TIMEZONE)
    return f"{agora.year}{agora.month:02d}{agora.day:02d}{agora.hour:02d}{agora.minute:02d}{agora.second:02d}"

def criar_contexto_ia(historico=[]):
    """Cria um contexto para a IA com informações sobre como deve atuar."""
    return {
        "role": "system",
        "content": """
Você é o atendente virtual da Neoenergia Elektro. Seu objetivo é identificar problemas de energia elétrica e decidir se deve abrir um chamado técnico.

DIRETRIZES IMPORTANTES:
1. Analise a mensagem do cliente para identificar o tipo de problema
2. Determine se o problema é interno (dentro da casa do cliente) ou externo (responsabilidade da Neoenergia)
3. Para problemas de falta de energia internos, primeiro oriente a verificar disjuntores do quadro interno e padrão de energia
4. Se o cliente relatar casa toda sem energia, pergunte se há LED aceso no medidor
5. Para problemas internos, oriente a procurar um eletricista particular
6. Para problemas que são responsabilidade da Neoenergia, confirme se o cliente deseja abrir um chamado técnico
7. Só gere protocolo APÓS a confirmação do cliente
8. Em casos de emergência (fios partidos, choques, etc), comece sua resposta com "ATENÇÃO!" e dê instruções de segurança
9. Seja objetivo, claro e direto - respostas curtas são preferíveis

FLUXO DE ATENDIMENTO:
1. Cumprimento inicial
2. Identificação do problema
3. Verificação se é interno ou externo
4. Para casos internos: orientação (disjuntores, eletricista)
5. Para casos externos: confirmação de abertura de chamado técnico
6. Geração de protocolo (apenas se confirmado)

ORDEM DOS PROCEDIMENTOS

1. A saudação é livre e muito simpática
2. Aguarde o cliente apresentar o problema
3. Questione e dê as orientações
4. Analise qual é o problema real
5. Apenas após isso aponte ao cliente qual é a possível falha
6. Se ofereça para gerar o chamado assim, Isso parece ser um problema de {problema['tipo']} - {problema['subtipo']}. Você quer que eu abra um chamado técnico?"
7. Aguarde os agradecimentos e despeça-se

Para problemas de emergência, priorize a segurança do cliente antes de qualquer procedimento.
"""
    }

def consultar_ia(mensagem_cliente, historico=[]):
    """Consulta a IA para obter resposta ao cliente baseada no contexto do atendimento."""
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    # Criar contexto inicial
    contexto_sistema = criar_contexto_ia()

    # Adicionar informações sobre categorias de problemas
    categorias_context = {
        "role": "system",
        "content": f"CATEGORIAS DE PROBLEMAS: {json.dumps(CATEGORIAS_PROBLEMAS, ensure_ascii=False)}"
    }

    # Montar mensagens para a IA
    messages = [contexto_sistema, categorias_context]

    # Adicionar histórico da conversa
    if historico:
        messages.extend(historico)

    # Adicionar mensagem atual do cliente
    messages.append({"role": "user", "content": mensagem_cliente})

    # Configurar parâmetros da API
    data = {
        "model": MODEL,
        "messages": messages,
        "temperature": 0.7,
        "max_tokens": 300  # Aumentado para permitir respostas mais completas
    }

    try:
        response = requests.post(URL, headers=headers, json=data)
        if response.status_code == 200:
            return response.json()["choices"][0]["message"]["content"]
        else:
            return usar_fallback(mensagem_cliente)
    except Exception as e:
        print(f"Erro ao consultar IA: {str(e)}")
        return usar_fallback(mensagem_cliente)

def usar_fallback(mensagem_cliente):
    """Fornece uma resposta de fallback baseada em análise direta da mensagem."""
    mensagem = mensagem_cliente.lower()

    # Verificar emergências primeiro
    for problema in CATEGORIAS_PROBLEMAS:
        if problema.get("criticidade") == "Emergência":
            for termo in problema.get("termos", []):
                if termo.lower() in mensagem:
                    return f"ATENÇÃO! Por segurança, mantenha-se afastado e não toque em nada. Identificamos um problema de {problema['tipo']} - {problema['subtipo']}. Você quer que eu abra um chamado técnico de emergência?"

    # Verificar problemas internos de falta de energia
    for problema in CATEGORIAS_PROBLEMAS:
        if problema.get("interno", False) and "Falta de Energia" in problema.get("tipo", ""):
            for termo in problema.get("termos", []):
                if termo.lower() in mensagem:
                    return "Entendi que você está sem energia. Primeiro, verifique os disjuntores do quadro interno e do padrão de energia. Há algum LED aceso no seu medidor?"

    # Verificar outros problemas
    for problema in CATEGORIAS_PROBLEMAS:
        for termo in problema.get("termos", []):
            if termo.lower() in mensagem:
                return f"Isso parece ser um problema de {problema['tipo']} - {problema['subtipo']}. Você quer que eu abra um chamado técnico?"

    # Caso não identifique o problema
    return "Poderia detalhar melhor o problema que está enfrentando? Assim posso ajudá-lo da melhor forma."

def processar_confirmacao_chamado(mensagem, historico):
    """Verifica se o cliente confirmou a abertura de chamado e gera protocolo."""
    confirmacoes = ["sim", "quero", "pode abrir", "abra", "preciso", "com certeza", "abre"]
    mensagem_lower = mensagem.lower()

    if any(termo in mensagem_lower for termo in confirmacoes):
        protocolo = gerar_protocolo()
        return f"Chamado técnico aberto com sucesso. Seu protocolo é {protocolo}. Nossa equipe irá resolver o problema o mais breve possível."

    negacoes = ["não", "nao", "não precisa", "nao precisa", "não quero", "nao quero"]
    if any(termo in mensagem_lower for termo in negacoes):
        return "Entendi que você não deseja abrir um chamado técnico. Caso precise de ajuda no futuro, estamos à disposição."

    # Se a resposta não for clara, consultar a IA novamente
    return None

def atendimento_cliente():
    """Função principal para iniciar atendimento."""
    print("\n=== Atendimento Neoenergia Elektro ===\n")

    historico = []
    primeira_mensagem = True
    esperando_confirmacao = False

    while True:
        if primeira_mensagem:
            saudacao = f"{obter_saudacao()}! Sou o Paulinho aqui da Neoenergia Elektro. Como posso ajudar?"
            print(f"[Atendente] {saudacao}")
            primeira_mensagem = False
            historico.append({"role": "assistant", "content": saudacao})

        mensagem_cliente = input("\n[Cliente] ").strip()

        # Verificar se é mensagem de despedida
        if mensagem_cliente.lower() in ["sair", "tchau", "encerrar", "obrigado", "agradeço"]:
            print(f"\n[Atendente] {mensagem_fechamento()}")
            break

        # Adicionar mensagem do cliente ao histórico
        historico.append({"role": "user", "content": mensagem_cliente})

        # Verificar se estamos esperando confirmação para abrir chamado
        if esperando_confirmacao:
            resposta_confirmacao = processar_confirmacao_chamado(mensagem_cliente, historico)
            if resposta_confirmacao:
                print(f"\n[Atendente] {resposta_confirmacao}")
                historico.append({"role": "assistant", "content": resposta_confirmacao})
                esperando_confirmacao = False
                continue

        # Consultar a IA para resposta
        resposta = consultar_ia(mensagem_cliente, historico)

        # Verificar se a resposta pede confirmação para abrir chamado
        if "você quer que eu abra um chamado técnico" in resposta.lower():
            esperando_confirmacao = True

        # Exibir e armazenar resposta no histórico
        print(f"\n[Atendente] {resposta}")
        historico.append({"role": "assistant", "content": resposta})

if __name__ == "__main__":
    atendimento_cliente()


=== Atendimento Neoenergia Elektro ===

[Atendente] Boa tarde! Sou o Paulinho aqui da Neoenergia Elektro. Como posso ajudar?

[Cliente] Olá, as lampadas aqui de casa não param de piscar

[Atendente] Desculpe ouvir isso! Isso pode ser um pouco desconfortável. Posso ajudar a identificar o problema. Você verifica se as luzes estão piscando em toda a casa ou só em alguns pontos específicos? Além disso, você notou se há algum outro sintoma, como aparelhos desligando ou ligando sozinhos?

[Cliente] notei que o ventilador também fica oscilando

[Atendente] Isso pode ser um indício de um problema de oscilação de tensão na sua rede elétrica. Posso ajudar a identificar se é um problema interno ou externo. Você verifica se o medidor de energia (o relógio de luz) está funcionando corretamente e se há algum LED aceso nele?

[Cliente] sim está normal tudo aceso 

[Atendente] Parece que o problema não está relacionado ao medidor de energia. Considerando que as luzes estão piscando e o ventilador est